# LM evaluation methods
- ref: chapter3 of SLP

## I. 评估语言模型的两类方法
1. 有两种评估类别：
   1. extrinsic evaluation: 针对具体任务跑结果，做对比。就具体任务而言，这种方式的结果更准确，但是耗时，成本高。
   2. intrinsic evaluation: 思路是，在训练集上训练，然后在测试集上评估模型质量。因此可以独立于任务来对模型做评估，但是在具体任务上并不是100%准确。

2. 语言模型的intrinsic evaluation: **<font color=orange>probability and perplexity</font>**
   1. 如何合理的划分训练集和测试集：
      - 为了防止多次使用test set，通常将数据集分成trainning set, dev set和test set。实践中，通常将数据按照8:1:1来划分。
      - 理论上：test set的大小至少要有足够的statistical power to measure a statistically significant difference between two models。
   2. **评价标准：**
      - **思路：**<font color=red>就两种语言模型的比较而言，在测试集上给出的probability更高的那个模型更好。</font>因为，给出的probability越高，意味着更好的预测了test set的出现是合理的。
      - **metric**：<font color=blue>实践中通常不会直接用probability，而是用perplexity。后者对文本长度做了调整，而且有更好的可解释性。</font>
   3. <font color=red>模型具有可比性的前提：训练时使用相同的vocabulary，测试时使用相同的test set。</font>

## II. Cross entropy
### II.1 几种cross entropy的定义
#### II.1.1 cross entropy的一般定义
   $$H(P,Q)=-E_PlogQ=-\sum_xP(x)logQ(x)$$

#### II.1.2 cross entropy of word
- 在给定language中，假设word的生成函数是$𝑃(x|context)$，则：
  $$\begin{align}
H(P(x|context),Q(x|context))& = -\sum_{c\in L}P(context=c)H(P(x|c),Q(x|c))\\
& = -\sum_{x,context\in L}P(x,context)logQ(x|context)
\end{align}
$$<font color=brown>[证明过程和entropy of word一样]</font>

#### II.1.3 per word cross entropy of a sequence
1. 符号
   - n个word构成的sequence记为$X=x_1,x_2,...,x_n$
   - sequence可以取language L中的任意长为N的sequence，记为$X\in L$
2. 定义：
   $$H_X(P,Q)=-\frac{1}{N}\sum_{x_1, ..., x_N\in L} P(x_1, x_2, ..., x_N)logQ(x_1, x_2, ..., x_N)$$
- 性质：$$H_X(P,Q) = -\frac{1}{N}\sum_{i=1}^N[H(P(x_i|x_{1:i-1}),Q(x_i|x_{1:i-1}))]$$
     - <font color=grey>证明：$$\begin{align}
H_X(P,Q) & = -\frac{1}{N}\sum_{X\in L} P(x_1, x_2, ..., x_N)logQ(x_1, x_2, ..., x_N) \\
& = -\frac{1}{N}\sum_{X\in L}P(x_1,x_2,...,x_N)\sum_{i=1}^NlogQ(x_i|x_1, ..., x_{i-1})\\
& = -\frac{1}{N}\sum_{i=1}^N\left[\sum_{X\in L}P(x_1,x_2,...,x_{\color{red} N} )logQ(x_i|x_1, ..., x_{i-1})\right]\\
& = -\frac{1}{N}\sum_{i=1}^N\left[\sum_{X\in L}P(x_1,x_2,...,x_{\color{red} i} )logQ(x_i|x_1, ..., x_{i-1})\right]\\
& = \frac{1}{N}[H(P(x_1),Q(x_1))+H(P(x_2|x_1),Q(x_2|x_1))+...+H(P(x_n|x_{1:n-1}),Q(x_n|x_{1:n-1}))]\\
\end{align}$$</font>

#### II.1.4 per word cross entropy of a language
   - 将语言看做可以用来生成infinite sequence的随机过程L，定义Language entropy：
   $$H_L(P,Q)=-\lim_{n\to \infty}\frac{1}{N}\sum_{x_1, ..., x_N\in L} P(x_1, x_2, ..., x_N)logQ(x_1, x_2, ..., x_N)$$

### II.2 sequence cross entropy的性质
#### II.2.1 per word cross entropy of the language
- 根据Shannon-McMillan-Breiman theorem有：<font color=brown>[ref:SLP,未证明]</font>
  $$\begin{align}
H_L(P,Q) & = \lim_{n\to \infty}-\frac{1}{N}\sum_{x_1, ..., x_N\in L} P(x_1, x_2, ..., x_N)logQ(x_1, x_2, ..., x_N)\\
& = \lim_{n\to \infty}-\frac{1}{N}\sum_{x_1, ..., x_N\in L}logQ(x_1, x_2, ..., x_N)
\end{align}$$
- 此时可以用样本sequence的统计特征来近似估计cross entropy。

#### II.2.2 per word cross entropy of sequences
- <font color=red>[提供了对上一条的证明]</font>
- 如果一个language对应的随机过程是ergodic且stationary的，那么用该随机过程对应的分布$P(x|context)$生成一个sequence X，再用model Q(x|context)去估计该sequence的生成概率所得到的per word cross entropy可以基于该语言所生成的一个sufficiently long sequence 样本{x_1, x_2, ..., x_N}来估计。
$$\begin{align}
\lim_{n\to \infty}H_X(P,Q)&=\lim_{n\to \infty}-\frac{1}{N}logQ(x_1, x_2, ..., x_N)\\
&=E_P[-logQ(x|context)]\\
&\approx -\frac{1}{N}logQ(x_1, x_2, ..., x_N)\\
\end{align}$$
  - <font color=red>注：这里$X\ne {x_1, x_2, ..., x_N}$。左边H(P,Q)是sequence X的分布P(X)和model Q(X)对应的cross entropy，X是随机变量序列而不是具体的某一个sequence，且其长度不一定是N。右边$\{x_1, x_2, ..., x_N\}$则是来自P(X)的一个长为N（sufficiently long）的sample sequence。</font>
  - <font color=green>[证明参考Perplexity note的附录]</font>

## III. Perplexity of a probability model

### III.1 从cross entropy引出Perplexity的定义
#### III.1.1 cross entropy的实践含义
1. 当x是单个随机变量时，如果x的实际分布是P(x)，但是却用Q(x)对event space中的events进行‘最优’(按Q(x)的分布的最优)编码，那么$H_x(P,Q)$就是最终得到的编码长度。
2. $H(P,Q)$可以用作模型评估的标准
   - 根据$H(P)与H_x(P,Q)$的关系：$$\begin{align}
& D_{KL}(P||Q)=H(P,Q)-H(P) \\
& \because D_{KL}\ge 0，iff P=Q时取等 \\
& \therefore H(P,Q)\ge H(P),iff P=Q时取等
\end{align}$$
   - $H(P)$是随机变量x的最优平均码长，而$H_x(P,Q)$是用模型Q估计P分布时得到的最优平均码长，只要$Q\ne P$，那么用Q编码的最优码长肯定大于用P编码得到的最优码长。
   - <font color=blue>假设有多个不同的估计模型$Q_1, Q_2$，那么谁的最优码长小，谁就与P更接近，因此是更好的model。**所以，H(P,Q)可以用作模型评估的标准。**</font>

#### III.1.2 Perplexity of probability model
1. **真实分布P(x)的Perplexity=$2^{H(x)}$**
   - $H(x)$是对分布进行编码能够实现的用bit计量的最短码长。
   - $2^{H(x)}$是effective branching factor，也称为effective number of equally probable choices。表达的含义是分布的熵等价于一个事件数量$|\mathcal{X}|=2^{H(x)}$的等概率分布的熵。
   - 所以，Perplexity本质上是和熵等价的另一种度量随机变量分布的不确定性的方式。
     - 熵用最优编码的bit数
     - Perplexity用<font color=norange>**等概率分布的事件数量**</font>

2. probability model Q(x)的Perplexity$2^{H_x(P,Q)}$
   - 用模型Q(x)来估计P(x)时，能够做到的平均最优编码bit数为${H_x(P,Q)}$，   因为$H(P,Q)\ge H(P),当且仅当P=Q时取等$，所以这个bit数一定大于真实分布的最优平均码长。
   - 假设有多个不同的估计模型$Q_1, Q_2$，那么谁的最优码长小，谁就与P更接近，因此是更好的model。**所以，H(P,Q)可以用作模型评估的标准。**
   - <font color=red>而machine learning领域，出于计算方便，用等价的另一种metric$2^{H_x(P,Q)}$来评估。因为在machine learning model中，常用的MLE目标函数中就包含了$2^{H_x(P,Q)}$的估计量。</font>并参考分布的Perplexity，将其定义为probability model的Perplexity。
   $$Perplexity(Q)=2^{H_x(P,Q)}$$

### III.2 Perplexity of language model
#### III.2.1 单个word
- 在给定language中，假设word的生成函数是$P(x|context)$，模型是$Q(x|context)$则：
  $$\begin{align}
H_{x|context}(P,Q)& = -\sum_{c\in L}P(context=c)H(P(x|c),Q(x|c))\\
& = -\sum_{x,context\in L}P(x,context)logQ(x|context)\\
Perplexity(Q) & =2^{H_{x|context}\,\,\,\,\,(P,Q)}
\end{align}
$$

#### III.2.2 language
1. 理想状态下的Perplexity of language model
   - 当语言的生成过程是ergodic且stationary时，有：
$$\begin{align}
H_L(P,Q) & =\lim_{n\to \infty}-\frac{1}{N}\sum_{x_1, ..., x_N\in L} P(x_1, x_2, ..., x_N)logQ(x_1, x_2, ..., x_N)\\
& =\lim_{n\to \infty}-\frac{1}{N}logQ(x_1, x_2, ..., x_N)\\
Perplexity(Q) & =2^{H_L\,(P,Q)}\\
\end{align}$$
   - 但实践中没有$N\to \infty$的sequence，因此只能用给定长的sequence来估计交叉熵。

2. 实践中使用的Perplexity of language model
   - 但实践中用test set作为sample sequence来估计language model的交叉熵，并将与之对应的$2^{H(P,Q)}$作为Perplexity of language model Q.
   $$\begin{align}
H_L(P,Q) & =\lim_{n\to \infty}-\frac{1}{N}\sum_{x_1, ..., x_N\in L} P(x_1, x_2, ..., x_N)logQ(x_1, x_2, ..., x_N)\\
& =\lim_{n\to \infty}-\frac{1}{N}logQ(x_1, x_2, ..., x_N)\\
& \approx -\frac{1}{N}logQ(x_1, x_2, ..., x_N)\\
Perplexity(Q) & =2^{H_L\,(P,Q)}\\
& \approx 2^{-\frac{1}{N}logQ(x_1, x_2, ..., x_N)} \\
& = Q(x_1, x_2, ..., x_N)^{-\frac{1}{N}}
\end{align}$$

#### III.2.3 用常用符号重写Perplexity
- <font color=red>实际在machine learning模型中不会使用$Q(x)$，而是直接用$P(x)$作为模型符号，为了与前面内容区分开，这里重新表述如下：</font>
- X为test set word sequence，N=|X|，有：
$$\begin{eqnarray*}
PPL(X) & = & P(x_1, x_2, ..., x_n)^{-1/N}  ...①\\
 & = & [ {\textstyle \prod_{i=1}^{N}}\ P(x_i|x_1:x_{i-1}) ]^{-1/N} ...②
\end{eqnarray*}$$

### III.3 Perplexity与probability的关系
- Perplexity与Probability of test sample sequence的一致性解释了：为什么要用Perplexity，而不是直接用H(P,Q)作为模型评估的指标
$$\begin{align}\underset{θ}{argmin}\ PPL(X;θ) & = \underset{θ}{argmax}\  P(X;θ)^{1/N} \\
           & = \underset{θ}{argmax}\ \frac{1}{N}  \ {\textstyle \sum_{i=1}^{N}}\ logP(y_{i}|X_{i};θ)\end{align}$$ 
   - <font color=orange>**在模型训练时，MLE的优化目标是找到: $\hat{θ}=argmaxP(X;θ)$。此时，$P(X;θ)$的值是多少不重要。** </font>
   - <font color=green>**而在模型评估时，则是将找到的以$\hat{θ}$为参数的模型用到test set上，看$P(X;θ)$的大小，用它来评价模型。** </font>